<h1>Задание 3<h1>

Собрать корпус документов по выбранной теме и по основы программ, разработанных в заданиях 1 и 2, подсчитать метрику TF-IDF для ключевых слов документа. (2 балла)

<h2>Лемматизация<h2>

In [9]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from pymystem3 import Mystem

#Create and stopwords list
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'как-то'])

mystem = Mystem() 

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [10]:
punctuation = punctuation + '...' + '—' + '«' + '»' + '``\n'
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~...—«»``\n'

In [11]:
def get_sentences(text):
    split_regex = re.compile(r'[.|!|?|…]')
    sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(text)])
    corpus = []
    for s in sentences:
        corpus.append(s)
    return(corpus)

In [12]:
#Preprocess function
def preprocess_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [ morph.parse(word.lower())[0].normal_form for word in tokens if (word != " ") and (word not in punctuation)]
    filtered_tokens = []
    for word in lemmatized_tokens:
        if (word not in russian_stopwords) and (word != " ") and (word not in punctuation):
            filtered_tokens.append(word)
    
    
    return filtered_tokens

In [13]:
from nltk import FreqDist
def frequency_of_words(txt):
    t = preprocess_text(txt)
    frequency_distribution = FreqDist(t)
    return frequency_distribution

In [14]:
def write_to_file(frequency_distribution, name):
    import csv
    name = str(name).replace('corpus_/','')
    path = 'words_frequency/' + name[:-4] + '.csv'
    with open(path, 'w') as csvfile:
        fieldnames = ['word', 'frequency']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for key in frequency_distribution:
             writer.writerow({'word': key, 'frequency': str(frequency_distribution[key])})
                                  
        
    csvfile.close()

    print("Writing complete")

In [15]:
import os
from pathlib import Path
source_dir = Path('corpus_/')
files = source_dir.glob('*.txt')
for file in files:
    txt = ''
    with file.open('r') as file_handle:
        for line in file_handle:
            txt += line.rstrip('\r\n') + ' '
    txt = get_sentences(txt)
    txt = ' '.join(txt)
    
    fr = frequency_of_words(txt)
    write_to_file(fr, file)
    print(file, ' done')

Writing complete
corpus_/Кино - Город.txt  done
Writing complete
corpus_/Кино - Электричка.txt  done
Writing complete
corpus_/Кино - Мои друзья.txt  done
Writing complete
corpus_/Кино - Разреши мне.txt  done
Writing complete
corpus_/Кино - В поисках сюжета.txt  done
Writing complete
corpus_/Кино - Малыш.txt  done
Writing complete
corpus_/Кино - Спокойная ночь.txt  done
Writing complete
corpus_/Кино - Группа крови.txt  done
Writing complete
corpus_/Кино - Пачка сигарет.txt  done
Writing complete
corpus_/Кино - Война.txt  done
Writing complete
corpus_/Кино - Место для шага вперед.txt  done
Writing complete
corpus_/Кино - Ночь.txt  done
Writing complete
corpus_/Кино - Верь мне.txt  done
Writing complete
corpus_/Кино - На кухне.txt  done
Writing complete
corpus_/Кино - Без десяти.txt  done
Writing complete
corpus_/Кино - Песня без слов.txt  done
Writing complete
corpus_/Кино - Вопрос.txt  done
Writing complete
corpus_/Кино - Ты обвела меня вокруг пальца.txt  done
Writing complete
corpus_/К

**TF термина а = (Количество раз, когда термин а встретился в тексте / количество всех слов в тексте)**

**IDF термина а = логарифм(Общее количество документов / Количество документов, в которых встречается термин а)**

**TF-IDF термина а = (TF термина а) * (IDF термина а)**

In [31]:
import pandas as pd

main_text = pd.read_csv('words_frequency/Кино - Перемен.csv')

all_words = main_text['frequency'].sum()
main_text['TF'] = main_text['frequency']/all_words

In [32]:
def compute_in_other_texts(word, corpus):
    return sum([1.0 for i in corpus if word in i])

def compute_idf(word, corpus):
        return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

In [33]:
import math
corpus = [0] * 68
source_dir = Path('words_frequency/')
files = source_dir.glob('*.csv')
main_list = list(main_text['word'])

main_text['in_other_texts'] = 0
main_text['IDF'] = 0

i=0
for file in files:
    text = pd.read_csv(file)
    text_list = list(text['word'])
    corpus[i] = text_list
    i+=1

i=0
for word in main_list:
    main_text.iloc[i,3] = compute_in_other_texts(word, corpus)
    i+=1

i=0
for word in main_list:
    main_text.iloc[i,4] = compute_idf(word, corpus)
    i+=1

main_text['TF-IDF'] = main_text['TF']*main_text['IDF']

In [34]:
main_text.shape

(63, 6)

если слово встречается в каком-либо документе часто, при этом встречаясь редко во всех остальных документах — это слово имеет большую значимость для того самого документа

In [35]:
main_text.sort_values('TF-IDF', ascending=False).head(60)

,word,frequency,TF,in_other_texts,IDF,TF-IDF
1,перемена,12,0.103448,1,1.832509,0.189570
2,требовать,6,0.051724,1,1.832509,0.094785
0,наш,13,0.112069,12,0.753328,0.084425
6,смех,3,0.025862,1,1.832509,0.047392
7,слеза,3,0.025862,1,1.832509,0.047392
8,пульсация,3,0.025862,1,1.832509,0.047392
9,вено,3,0.025862,1,1.832509,0.047392
5,сердце,3,0.025862,3,1.355388,0.035053
3,глаз,4,0.034483,10,0.832509,0.028707
12,вместо,2,0.017241,2,1.531479,0.026405
